In [1]:
from scripts.dashboard import Dashboard

ds = Dashboard(graphs=False)

Leyendo datos de web lvera: 100%|██████████| 22/22 [00:00<00:00, 105.59it/s]

Datos de lvera listos.


In [2]:
import os
import pandas as pd
import xlsxwriter as xw
from scripts.calculations import Utils
from scripts.calculations import Anomalies
from dateutil.relativedelta import relativedelta

In [27]:
from scripts.calculations import LastData

In [31]:
df = ds.last_data_df.reset_index()

In [33]:
ds.year

2023

In [3]:
if ds.day == 1:
    this_month = ds.past_month
    month = (ds.today - relativedelta(months=1)).strftime("%b")
    this_month_day_list = ds.past_month_days_list
else:
    this_month = ds.this_month
    month = ds.today.strftime("%b")
    this_month_day_list = ds.this_month_days_list

In [5]:
month = ds.today.strftime("%b")
exported_path = "Data/Exported"

# Lista de estaciones pluviométricas
plu_st = ["LIVES", "CHUGUR"]

In [6]:
# Creando hoja de excel
wbk = xw.Workbook(os.path.join(exported_path, f"{this_month}_{month}_DATOS.xlsx"))

In [7]:
# Hoja con promedios y acumulados mensuales
wst_means = wbk.add_worksheet("Means")

# Calculo
tmin_mean = ds.get_monthly_means("tmin", ds.mn, ds.this_month).rename(
    columns={"Mean": "TMIN"}
)
tmax_mean = ds.get_monthly_means("tmax", ds.mx, this_month).rename(
    columns={"Mean": "TMAX"}
)
pp_sum = ds.get_monthly_means("pp", ds.pp, this_month).rename(columns={"Sum": "PP"})

# Combinando datos en un solo dataframe
df_means = pd.DataFrame({
    'TMAX': tmax_mean['TMAX'],
    'TMIN': tmin_mean['TMIN'],
    'PP': pp_sum['PP']
}).reindex(columns=['TMAX', 'TMIN', 'PP'])

# Añadiendo coordenadas
df = Utils(df_means, ds.file).add_lat_lon().rename_axis("Estaciones").reset_index()

#data_wo_color(wbk, wst_means, df)
ExcelWriter(wbk, wst_means).no_color_cell(df)

In [8]:
wst_anomalies = wbk.add_worksheet("Anomalies")

amn = Anomalies(
    "tmin", tmin_mean, ds.this_month, ds.file, ds.file_w_normals
).calculate_anomalies()
amx = Anomalies(
    "tmax", tmax_mean, this_month, ds.file, ds.file_w_normals
).calculate_anomalies()
app = Anomalies(
    "pp", pp_sum, this_month, ds.file, ds.file_w_normals
).calculate_anomalies()

df_anomalies = pd.DataFrame({
    'TMAX': amx['anomaly'],
    'TMIN': amn['anomaly'],
    'PP': app['anomaly']
}).reindex(columns=['TMAX', 'TMIN', 'PP'])

df = Utils(df_anomalies, ds.file).add_lat_lon().rename_axis("Estaciones").reset_index()

#data_wo_color(wbk, wst_anomalies, df)
ExcelWriter(wbk, wst_anomalies).no_color_cell(df)

In [9]:
# Hoja de datos diarios de TMAX (todas las estaciones)
wst_tmax = wbk.add_worksheet("TMAX")

# Extrayendo los datos del mes actual
df_tmax = ds.mx[ds.mx.index.month == this_month]

df = pd.DataFrame(index=this_month_day_list, columns=df_tmax.columns)

for i in range(len(df_tmax)):
    df.iloc[i] = df_tmax.values[i]

df = df.T[~df.T.index.isin(plu_st)]
df = df.rename_axis("Estaciones").reset_index()

#data_wo_color(wbk, wst_tmax, df, extend=True)
ExcelWriter(wbk, wst_tmax).no_color_cell(df, extend=True)

In [10]:
# Hoja de datos diarios de TMIN (todas las estaciones)
wst_tmin = wbk.add_worksheet("TMIN")

# Extrayendo los datos del mes actual
df_tmin = ds.mn[ds.mn.index.month == ds.this_month]

df = pd.DataFrame(index=ds.this_month_days_list, columns=df_tmin.columns)

for i in range(len(df_tmin)):
    df.iloc[i] = df_tmin.values[i]

df = df.T[~df.T.index.isin(plu_st)]
df = df.rename_axis("Estaciones").reset_index()

#data_wo_color(wbk, wst_tmin, df, extend=True)
ExcelWriter(wbk, wst_tmin).no_color_cell(df, extend=True)

In [11]:
# Hoja de datos diarios de PP (todas las estaciones)
wst_pp = wbk.add_worksheet("PP")

# Extrayendo los datos del mes actual
df_pp = ds.pp[ds.pp.index.month == this_month]

df = pd.DataFrame(index=this_month_day_list, columns=df_pp.columns)

for i in range(len(df_pp)):
    df.iloc[i] = df_pp.values[i]

df = df.T.rename_axis("Estaciones").reset_index()

#data_wo_color(wbk, wst_pp, df, extend=True)
ExcelWriter(wbk, wst_pp).no_color_cell(df, extend=True)

In [12]:
# Hoja de DECADIARIAS
wst_dec = wbk.add_worksheet("Dec")

from scripts.calculations import Decadiarias

df = Decadiarias(ds.pp, this_month, ds.file, ds.dec_file).calculate_anom().iloc[:, :-3]

#data_wo_color(wbk, wst_dec, df)

ExcelWriter(wbk, wst_dec).no_color_cell(df)

In [13]:
# Hoja de caracterización de TMAX
wst_ctmax = wbk.add_worksheet("cTmax")
#data_w_color(wbk, wst_ctmax, 'tmax', ds, this_month, this_month_day_list)
ExcelWriter(wbk, wst_ctmax).colored_cell('tmax', ds, this_month, this_month_day_list)

In [14]:
# Hoja de caracterización de TMIN
wst_ctmin = wbk.add_worksheet("cTmin")
#data_w_color(wbk, wst_ctmin, 'tmin', ds, this_month, this_month_day_list)
ExcelWriter(wbk, wst_ctmin).colored_cell('tmin', ds, this_month, this_month_day_list)

In [15]:
# Hoja de caracterización de PP
wst_cpp = wbk.add_worksheet("cPP")
#data_w_color(wbk, wst_cpp, 'pp', ds, this_month, this_month_day_list)
ExcelWriter(wbk, wst_cpp).colored_cell('pp', ds, this_month, this_month_day_list)

In [16]:
wbk.close()